<a href="https://colab.research.google.com/github/cannin/gsoc_2023_pytorch_pathway_commons/blob/main/Building_GNNmodel_with_PyG.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install torch-geometric

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 661.6/661.6 kB 34.1 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for torch-geometric: filename=torch_geometric-2.3.1-py3-none-any.whl size=910459 sha256=94f2b4f64eda9becfd8866dbe0fd3a60aa9a0f175fd5595663c82d5eeab9a051
  Stored in directory: /root/.cache/pip/wheels/ac/dc/30/e2874821ff308ee67dcd7a66dbde912411e19e35a1addda028
Successfully built torch-geometric


In [10]:
import pandas as pd
import numpy as np
import torch
from torch_geometric.data import Data

In [23]:
cBio = pd.read_csv('/content/cBio (1).csv')

In [24]:
cBio.head()

,Sample Identifier,Overall Survival (Months),NUDT18,SHC3,NCRNA00112,FAM49B,FMO6P,BAT2L2,SIRT1,MSLNL,...,SCARNA6,SNORD77,TLR10,GKN2,PCDHGB4,ZMAT5,AXIN2,LONRF1,KLHL10,C12orf36
0,TCGA-OR-A5J1-01,44.547457,85.0144,97.5024,0.0,671.950,0.0,2428.43,462.056,0.0,...,0.0000,0.0,12.0077,0.0,17.2911,913.535,547.5500,363.112,1.9212,0.0
1,TCGA-OR-A5J2-01,55.133642,165.8840,92.6050,0.0,812.911,0.0,5097.30,669.977,0.0,...,0.4026,0.0,0.8053,0.0,268.9890,303.978,1793.3200,592.672,0.0000,0.0
2,TCGA-OR-A5J3-01,68.744452,309.2880,10.0726,0.0,482.299,0.0,1615.76,514.294,0.0,...,0.5925,0.0,0.0000,0.0,689.4860,324.592,522.5890,456.821,0.0000,0.0
3,TCGA-OR-A5J5-01,11.999868,178.9310,49.5740,0.0,466.305,0.0,1202.17,591.789,0.0,...,0.0000,0.0,6.9713,0.0,54.2215,790.550,337.7230,484.895,1.5492,0.0
4,TCGA-OR-A5J6-01,88.864780,124.0660,5.5886,0.0,1074.680,0.0,1424.53,805.312,0.0,...,1.6766,0.0,1.1177,0.0,8.9417,291.550,9.5006,628.155,1.1177,0.0


In [14]:
path = "/content/drive/MyDrive/gsoc data/PathwayCommons12.reactome.hgnc.sif.gz"  


#specify compression type because the file is compressed
df = pd.read_csv(path, sep="\t", compression="gzip", header=None, names=["Source", "InteractionType", "Target"])

In [15]:
df.head()

,Source,InteractionType,Target
0,A1CF,in-complex-with,APOBEC1
1,A1CF,in-complex-with,APOBEC2
2,A1CF,in-complex-with,APOBEC3A
3,A1CF,in-complex-with,APOBEC3B
4,A1CF,in-complex-with,APOBEC3C


In [16]:
reduced_df = df.drop(columns='InteractionType')

In [17]:
source_nodes = reduced_df['Source'].tolist()
target_nodes = reduced_df['Target'].tolist()

# Create a set of all unique nodes
all_nodes = set(source_nodes + target_nodes)

# Create a dictionary to map each unique node to a unique index
node_to_index = {node: index for index, node in enumerate(all_nodes)}

# Map the source and target nodes to their corresponding indices
source_indices = [node_to_index[node] for node in source_nodes]
target_indices = [node_to_index[node] for node in target_nodes]

In [18]:
# Convert the source and target indices to a PyTorch tensor
edge_index = torch.tensor([source_indices, target_indices], dtype=torch.long)

# Create the graph data object
graph_data = Data(edge_index=edge_index)

In [25]:
node_features = cBio.iloc[:, 2:].values  # Extract the node feature values

num_nodes, num_features = node_features.shape  # Get the shape of the node features

# Create an empty array to store the node features
x = np.zeros((num_nodes, num_features), dtype=np.float32)

# Fill in the array with the node feature values
for i, row in cBio.iterrows():
    node_id = row['Sample Identifier']
    if node_id in node_to_index:  # Check if the node is present in the graph structure
        node_index = node_to_index[node_id]
        x[node_index] = row.values[2:]  # Exclude the Sample Identifier and Overall Survival columns

# Convert the array to a PyTorch tensor
x = torch.from_numpy(x)

# Create a PyG Data object with the graph structure and node features
data = Data(x=x, edge_index=edge_index)

In [26]:
print(data)

Data(x=[78, 20511], edge_index=[2, 353636])


In [28]:
print(data.num_nodes)  # Print the number of nodes
print(data.num_edges)  # Print the number of edges

78
353636


In [29]:
print(data.x)  # Print the node features

tensor([[0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        ...,
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.]])
